In [1]:
import librosa
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join, basename, splitext
import ast

In [2]:
def load_metadata(filepath):
    filename = basename(filepath)
    
    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)
    
    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

    COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
               ('track', 'genres'), ('track', 'genres_all')]
    for column in COLUMNS:
        tracks[column] = tracks[column].map(ast.literal_eval)

    COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
               ('album', 'date_created'), ('album', 'date_released'),
               ('artist', 'date_created'), ('artist', 'active_year_begin'),
               ('artist', 'active_year_end')]
    for column in COLUMNS:
        tracks[column] = pd.to_datetime(tracks[column])

    SUBSETS = ('small', 'medium', 'large')
    tracks['set', 'subset'] = tracks['set', 'subset'].astype(
            'category', categories=SUBSETS, ordered=True)

    COLUMNS = [('track', 'license'), ('artist', 'bio'),
               ('album', 'type'), ('album', 'information')]
    for column in COLUMNS:
        tracks[column] = tracks[column].astype('category')

    return tracks

In [3]:
def extract(track_metadata, genre_metadata, path):
    id = 0
    dataset = []
    mp3_files = librosa.util.find_files(path) # load all mp3 files in the path, including inside subdirectories
        
    for i in xrange(len(mp3_files)):
        # for some reason, librosa's find_file returns two of each file in a path, and we don't want that
        if i % 2 == 0:
            features = []

            id = id + 1

            print("Reading Song#{}: ".format(id) + mp3_files[i] + "...")

            # Features related to music, such as beat, tempo, etc
            y, sr = librosa.load(mp3_files[i], duration=30)
            tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            rmse = librosa.feature.rmse(y=y)
            cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            genre = get_genre_for_current_song(track_metadata, genre_metadata, mp3_files[i])
            print genre

            # appending all the features to a list
            features.append(id)
            features.append(mp3_files[i])
            features.append(tempo)
            features.append(np.sum(beats))
            features.append(np.mean(chroma_stft))
            features.append(np.mean(rmse))
            features.append(np.mean(cent))
            features.append(np.mean(spec_bw))
            features.append(np.mean(rolloff))
            features.append(np.mean(zcr))
            for coefficient in mfcc:
                features.append(np.mean(coefficient))
            features.append(genre)

            # appending the list of features to another list for our dataset
            dataset.append(features)
    return dataset

In [4]:
def get_genre_for_current_song(track_metadata, genre_metadata, song):
    
    # Formatting file name to leave out the filepath and extension, as well as remove leading 0's
    song_name = basename(song)
    song_name= splitext(song_name)[0]
    song_name = song_name.lstrip('0')
    song_no = int(song_name)
    
    # Checking to make sure that the genre value isn't nan
    if track_metadata.loc[song_no][("track", "genre_top")] == track_metadata.loc[song_no][("track", "genre_top")]:
        return track_metadata.loc[song_no][("track", "genre_top")] # return the genre
    # the top genre value was nan, so we find the song's most popular genre manually
    else:
        num_tracks = 0
        genre_index = 0
        genre_list = track_metadata.loc[song_no][("track", "genres")]
        for i in xrange(len(genre_list)):
            if genres.loc[genre_list[i]]["#tracks"] > num_tracks:
                num_tracks = genres.loc[genre_list[i]]["#tracks"]
                genre_index = i
        return genre_metadata.loc[genre_list[index]]["title"]

In [5]:
AUDIO_DIR = "C:\\Users\\1harr\\Desktop\\Fma_data\\fma_test" # directory for fma music

# location of the track metadata that came along with the fma dataset
TRACK_METADATA = "C:\\Users\\1harr\\Desktop\\Fma_data\\fma_metadata\\tracks.csv" 

# location of the genre metadata that came along with the fma dataset
GENRE_METADATA = "C:\\Users\\1harr\\Desktop\\Fma_data\\fma_metadata\\genres.csv"

In [6]:
tracks = load_metadata(TRACK_METADATA)
genres = load_metadata(GENRE_METADATA)

C:\Users\1harr\Anaconda2\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


In [7]:
features = extract(tracks, genres, AUDIO_DIR)

Reading Song#1: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000002.mp3...
Hip-Hop
Reading Song#2: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000005.mp3...
Hip-Hop
Reading Song#3: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000010.mp3...
Pop
Reading Song#4: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000140.mp3...
Folk
Reading Song#5: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000141.mp3...
Folk
Reading Song#6: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000148.mp3...
Experimental
Reading Song#7: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000182.mp3...
Rock
Reading Song#8: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000190.mp3...
Folk
Reading Song#9: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000193.mp3...
Folk
Reading Song#10: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000194.mp3...
Folk
Reading Song#11: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000197.mp3...
Folk
Reading Song#12: C:\Users\1harr\Desktop\Fma_data\fma_test\000\000200.mp3...
Folk
Reading Song#13: C:\User

International
Reading Song#99: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001683.mp3...
International
Reading Song#100: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001684.mp3...
International
Reading Song#101: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001685.mp3...
International
Reading Song#102: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001686.mp3...
International
Reading Song#103: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001687.mp3...
International
Reading Song#104: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001688.mp3...
International
Reading Song#105: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001689.mp3...
International
Reading Song#106: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001701.mp3...
Folk
Reading Song#107: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001702.mp3...
Folk
Reading Song#108: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001703.mp3...
Folk
Reading Song#109: C:\Users\1harr\Desktop\Fma_data\fma_test\001\001704.mp3...
Folk
Reading Song#110: C:\U

In [8]:
heading = ['id', 'songname', 'tempo', 'beats', 'chromagram', 'rmse',
           'centroid', 'bandwidth', 'rolloff', 'zcr', 'mfcc1', 'mfcc2',
           'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9',
           'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15',
           'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'genre']

In [11]:
df = pd.DataFrame(features, columns=heading)
df

,id,songname,tempo,beats,chromagram,rmse,centroid,bandwidth,rolloff,zcr,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,genre
0,1,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,161.499023,50909,0.396905,0.145191,3037.646484,2690.117840,6214.680908,0.163720,...,3.406905,-3.014135,8.196358,-3.306116,1.253568,-8.481506,-2.041248,-2.946169,0.093848,Hip-Hop
1,2,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,99.384014,31239,0.424453,0.148829,2414.503249,2598.633925,5437.167122,0.100103,...,8.863636,3.440012,7.301098,0.220522,0.931993,-4.121887,-0.581281,-1.604948,1.341656,Hip-Hop
2,3,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,112.347147,34889,0.397663,0.188248,2349.564320,2051.910358,4315.847696,0.148673,...,2.240332,-6.418429,2.678393,-5.106335,-0.910397,-3.209620,3.044798,-1.036912,3.884589,Pop
3,4,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,107.666016,32545,0.295095,0.069216,1746.271125,2484.461469,3863.058309,0.044345,...,-1.614263,-6.837076,2.880230,-8.271350,1.238807,-5.206714,-0.754464,-6.319953,-1.172449,Folk
4,5,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,103.359375,31837,0.262046,0.101356,1654.240257,2036.525423,3315.662935,0.061877,...,-3.396589,-4.810521,3.130622,-2.126631,5.054061,-0.169659,3.087798,-8.665619,-1.098305,Folk
5,6,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,117.453835,20944,0.332138,0.478672,1707.170372,1878.780098,3543.246042,0.090190,...,7.165055,-0.238565,10.885093,5.995166,6.319376,-1.196675,8.379917,8.224323,4.861701,Experimental
6,7,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,143.554688,42774,0.369365,0.275298,1876.797169,1749.208960,3949.559451,0.100565,...,12.425987,3.194551,11.543197,-2.851272,1.792177,-0.097796,-1.911920,-0.754154,-1.609499,Rock
7,8,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,99.384014,32062,0.244967,0.132980,1899.920558,2019.028214,3715.077535,0.088393,...,-4.756800,-8.715330,-6.131213,-8.163444,-8.739534,-9.175488,-2.000137,-6.302590,-8.047442,Folk
8,9,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,95.703125,28646,0.335681,0.083325,1584.312861,1754.518587,3207.379779,0.077351,...,-4.999176,-7.849339,-7.091806,-13.875962,-8.471770,-10.473596,-8.710501,-9.456114,-9.770329,Folk
9,10,C:\Users\1harr\Desktop\Fma_data\fma_test\000\0...,103.359375,32113,0.307242,0.096473,1546.779400,1760.761918,3087.247999,0.072886,...,-2.445210,-8.818468,-9.325425,-9.184675,-5.520606,-10.009364,-6.689494,-8.279938,-6.850169,Folk


In [12]:
df.ix[:, "tempo":]

C:\Users\1harr\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,tempo,beats,chromagram,rmse,centroid,bandwidth,rolloff,zcr,mfcc1,mfcc2,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,genre
0,161.499023,50909,0.396905,0.145191,3037.646484,2690.117840,6214.680908,0.163720,-67.385072,65.155028,...,3.406905,-3.014135,8.196358,-3.306116,1.253568,-8.481506,-2.041248,-2.946169,0.093848,Hip-Hop
1,99.384014,31239,0.424453,0.148829,2414.503249,2598.633925,5437.167122,0.100103,-106.550053,87.231814,...,8.863636,3.440012,7.301098,0.220522,0.931993,-4.121887,-0.581281,-1.604948,1.341656,Hip-Hop
2,112.347147,34889,0.397663,0.188248,2349.564320,2051.910358,4315.847696,0.148673,-17.240730,94.442728,...,2.240332,-6.418429,2.678393,-5.106335,-0.910397,-3.209620,3.044798,-1.036912,3.884589,Pop
3,107.666016,32545,0.295095,0.069216,1746.271125,2484.461469,3863.058309,0.044345,-278.577023,101.531942,...,-1.614263,-6.837076,2.880230,-8.271350,1.238807,-5.206714,-0.754464,-6.319953,-1.172449,Folk
4,103.359375,31837,0.262046,0.101356,1654.240257,2036.525423,3315.662935,0.061877,-208.569572,116.634597,...,-3.396589,-4.810521,3.130622,-2.126631,5.054061,-0.169659,3.087798,-8.665619,-1.098305,Folk
5,117.453835,20944,0.332138,0.478672,1707.170372,1878.780098,3543.246042,0.090190,7.706248,126.735480,...,7.165055,-0.238565,10.885093,5.995166,6.319376,-1.196675,8.379917,8.224323,4.861701,Experimental
6,143.554688,42774,0.369365,0.275298,1876.797169,1749.208960,3949.559451,0.100565,-26.018376,136.665445,...,12.425987,3.194551,11.543197,-2.851272,1.792177,-0.097796,-1.911920,-0.754154,-1.609499,Rock
7,99.384014,32062,0.244967,0.132980,1899.920558,2019.028214,3715.077535,0.088393,-139.908389,97.182367,...,-4.756800,-8.715330,-6.131213,-8.163444,-8.739534,-9.175488,-2.000137,-6.302590,-8.047442,Folk
8,95.703125,28646,0.335681,0.083325,1584.312861,1754.518587,3207.379779,0.077351,-198.296480,120.712114,...,-4.999176,-7.849339,-7.091806,-13.875962,-8.471770,-10.473596,-8.710501,-9.456114,-9.770329,Folk
9,103.359375,32113,0.307242,0.096473,1546.779400,1760.761918,3087.247999,0.072886,-184.892418,119.175392,...,-2.445210,-8.818468,-9.325425,-9.184675,-5.520606,-10.009364,-6.689494,-8.279938,-6.850169,Folk
